In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
toxprot_17 = pd.read_csv("../data/processed/toxprot_2017.csv")
toxprot_25 = pd.read_csv("../data/processed/toxprot_2025.csv")

In [ ]:
toxprot_25["Protein families"].value_counts()

In [ ]:
toxprot_17["Fragment"].value_counts(dropna=False)

In [ ]:
toxprot_25["Protein families"].value_counts()

### Figures

In [ ]:
bins = np.arange(1, 325, 25)

bin_labels = [f"{start}-{end - 1}" for start, end in zip(bins[:-1], bins[1:])]
bin_labels[-1] = "300+"

lengths_25 = toxprot_25["Length"]
lengths_17 = toxprot_17["Length"]

plt.figure(figsize=(12, 8))
plt.hist(lengths_25, bins=bins, color=plt.cm.tab10(1), label="toxprot_25")
plt.hist(lengths_17, bins=bins, color=plt.cm.tab10(0), label="toxprot_17")

plt.xlabel("Length Bins")
plt.ylabel("Count")
plt.title("Overlaid Histogram of Sequence Lengths")

plt.xticks(bins[:-1] + 12.5, bin_labels, fontsize=9)

plt.legend()
plt.savefig("../figures/overlaid_hist.png", dpi=300)

plt.show()
plt.close()

In [ ]:
toxprot_17["Fragment"].value_counts()

In [ ]:
na_count_17 = toxprot_17["Protein families"].isna().sum()  # Count NaN values
total_count_17 = len(toxprot_17["Protein families"])  # Total number of entries
na_percentage_17 = (
    (na_count_17 / total_count_17) * 100 if total_count_17 > 0 else 0
)  # Calculate percentage
unique_families_17 = len(toxprot_17["Protein families"].dropna().unique())

print(f"N/A (NaN) Count: {na_count_17}")
print(f"{unique_families_17}")
print(f"N/A (NaN) Percentage: {na_percentage_17:.2f}%")

na_count_25 = toxprot_25["Protein families"].isna().sum()  # Count NaN values
total_count_25 = len(toxprot_25["Protein families"])  # Total number of entries
na_percentage_25 = (
    (na_count_25 / total_count_25) * 100 if total_count_25 > 0 else 0
)  # Calculate percentage
unique_families_25 = len(toxprot_25["Protein families"].dropna().unique())

print(f"N/A (NaN) Count: {na_count_25}")
print(f"{unique_families_25}")
print(f"N/A (NaN) Percentage: {na_percentage_25:.2f}%")


In [ ]:
def get_top_families_per_dataset(df, column, top_n=15):
    counts = df[column].value_counts()
    top_counts = counts.nlargest(top_n)
    other_count = counts.iloc[top_n:].sum()
    nan_count = df[column].isna().sum()

    top_counts["Other"] = other_count
    if nan_count > 0:
        top_counts["NaN"] = nan_count
    return top_counts


# Get top families per dataset
top_17 = get_top_families_per_dataset(toxprot_17, "Protein families", top_n=15)
top_25 = get_top_families_per_dataset(toxprot_25, "Protein families", top_n=15)

# Build unified index
top_17_sorted = top_17.sort_values(ascending=False)
top_25_sorted = top_25.sort_values(ascending=False)
all_families = top_17_sorted.index.to_list()
for fam in top_25_sorted.index:
    if fam not in all_families:
        all_families.append(fam)

# Create DataFrames
df_stacked = pd.DataFrame(index=all_families)
df_stacked["toxprot_17"] = top_17.reindex(all_families).fillna(0)
df_stacked["toxprot_25"] = top_25.reindex(all_families).fillna(0)
df_stacked_perc = df_stacked.div(df_stacked.sum()) * 100

# Colors
colors = plt.colormaps["tab20"]
families_no_nan_other = [f for f in all_families if f not in ("Other", "NaN")]
color_map = {fam: colors(i) for i, fam in enumerate(families_no_nan_other)}
color_map["Other"] = colors(18)
color_map["NaN"] = colors(19)
if "Huwentoxin-1 family" in color_map:
    color_map["Neurotoxin 10 (Hwtx-1) family"] = color_map["Huwentoxin-1 family"]
if "Spider toxin CSTX superfamily" in color_map:
    color_map["Neurotoxin 19 (CSTX) family"] = color_map[
        "Spider toxin CSTX superfamily"
    ]

# Plot
fig, ax = plt.subplots(figsize=(13, 9))

# Bar 1 (toxprot_17)
bottom = 0
order_17 = df_stacked["toxprot_17"].sort_values(ascending=False).index
bars17 = []
for fam in order_17:
    pct = df_stacked_perc.at[fam, "toxprot_17"]
    cnt = df_stacked.at[fam, "toxprot_17"]
    if cnt > 0:
        bar = ax.bar(
            "toxprot_17", pct, bottom=bottom, color=color_map.get(fam, "gray")
        )[0]
        bars17.append((bar, fam, cnt, pct))
    bottom += pct

# Bar 2 (toxprot_25)
bottom = 0
order_25 = df_stacked["toxprot_25"].sort_values(ascending=False).index
bars25 = []
for fam in order_25:
    pct = df_stacked_perc.at[fam, "toxprot_25"]
    cnt = df_stacked.at[fam, "toxprot_25"]
    if cnt > 0:
        bar = ax.bar(
            "toxprot_25", pct, bottom=bottom, color=color_map.get(fam, "gray")
        )[0]
        bars25.append((bar, fam, cnt, pct))
    bottom += pct

# Add labels using each bar’s .get_x()/.get_width()
for bar, fam, cnt, pct in bars17 + bars25:
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_y() + bar.get_height() / 2,
        f"{fam}, {int(cnt)} ({pct:.1f}%)",
        ha="center",
        va="center",
        fontsize=9,
        wrap=True,
    )

# Final formatting
ax.set_ylabel("Percentage")
ax.set_title("100% Stacked Bar Chart of Protein Families")
ax.set_ylim(0, 100)
ax.tick_params(axis="x", labelsize=10)
ax.tick_params(axis="y", labelsize=10)

plt.tight_layout()
plt.savefig("../figures/stacked_bar_protfam.png", dpi=300)
plt.show()
plt.close()

In [ ]:
# Ensure the 'Fragment' column is treated as a string
toxprot_17["Fragment"] = toxprot_17["Fragment"].astype(str)
toxprot_25["Fragment"] = toxprot_25["Fragment"].astype(str)

# Count occurrences of "fragment" (case-insensitive)
fragment_count_17 = (
    toxprot_17["Fragment"].str.contains("fragment", case=False, na=False).sum()
)
total_count_17 = toxprot_17.shape[0]
fragment_percentage_17 = (
    (fragment_count_17 / total_count_17) * 100 if total_count_17 > 0 else 0
)

print(f"Fragment Count 17: {fragment_count_17}")
print(f"Fragment Percentage 17: {fragment_percentage_17:.2f}%")

fragment_count_25 = (
    toxprot_25["Fragment"].str.contains("fragment", case=False, na=False).sum()
)
total_count_25 = toxprot_25.shape[0]
fragment_percentage_25 = (
    (fragment_count_25 / total_count_25) * 100 if total_count_25 > 0 else 0
)

print(f"Fragment Count 25: {fragment_count_25}")
print(f"Fragment Percentage 25: {fragment_percentage_25:.2f}%")

In [ ]:
# Count occurrences where PTM is not NaN
ptm_count_17 = toxprot_17["PTM"].notna().sum()
ptm_percentage_17 = (ptm_count_17 / total_count_17) * 100 if total_count_17 > 0 else 0

print(f"ptm Count (2017): {ptm_count_17}")
print(f"ptm Percentage (2017): {ptm_percentage_17:.2f}%")

# Count occurrences where PTM is not NaN in 2025 datap
ptm_count_25 = toxprot_25["PTM"].notna().sum()
ptm_percentage_25 = (ptm_count_25 / total_count_25) * 100 if total_count_25 > 0 else 0

print(f"ptm Count (2025): {ptm_count_25}")
print(f"ptm Percentage (2025): {ptm_percentage_25:.2f}%")

In [ ]:
# Count occurrences where toxic is not NaN
toxic_count_17 = toxprot_17["Toxic dose"].notna().sum()
toxic_percentage_17 = (
    (toxic_count_17 / total_count_17) * 100 if total_count_17 > 0 else 0
)

print(f"toxic Count (2017): {toxic_count_17}")
print(f"toxic Percentage (2017): {toxic_percentage_17:.2f}%")

# Count occurrences where toxic is not NaN in 2025 data
toxic_count_25 = toxprot_25["Toxic dose"].notna().sum()
toxic_percentage_25 = (
    (toxic_count_25 / total_count_25) * 100 if total_count_25 > 0 else 0
)

print(f"Toxic dose Count (2025): {toxic_count_25}")
print(f"PTM Percentage (2025): {toxic_percentage_25:.2f}%")

In [ ]:
len(toxprot_17["Species"].value_counts())

In [ ]:
cell_text = [
    [total_count_17, total_count_25],
    [unique_families_17, unique_families_25],
    [
        f"{na_count_17} ({na_percentage_17:.2f}%)",
        f"{na_count_25} ({na_percentage_25:.2f}%)",
    ],
    [
        f"{fragment_count_17} ({fragment_percentage_17:.2f}%)",
        f"{fragment_count_25} ({fragment_percentage_25:.2f}%)",
    ],
    [
        f"{ptm_count_17} ({ptm_percentage_17:.2f}%)",
        f"{ptm_count_25} ({ptm_percentage_25:.2f}%)",
    ],
    [
        f"{toxic_count_17} ({toxic_percentage_17:.2f}%)",
        f"{toxic_count_25} ({toxic_percentage_25:.2f}%)",
    ],
    [
        len(toxprot_17["Species"].unique()),
        len(toxprot_25["Species"].unique()),
    ],
    [len(toxprot_17["Order"].unique()), len(toxprot_25["Order"].unique())],
]
row_headers = [
    "Total entry count",
    'unique Protein families count\n(collapsed after first ",")',
    "N/A protein family count",
    "Entries that are Fragments",
    "Entries with PTM annotation",
    "Entries with toxic dose annotation",
    "Number of species",
    "Number of order",
]
column_headers = ["2017", "2025"]

rcolors = plt.cm.BuPu(np.full(len(row_headers), 0.1))
ccolors = plt.cm.BuPu(np.full(len(column_headers), 0.1))

fig, ax = plt.subplots(figsize=(12, 6))
ax.axis("tight")
ax.axis("off")

the_table = plt.table(
    cellText=cell_text,
    rowLabels=row_headers,
    rowColours=rcolors,
    rowLoc="center",
    colColours=ccolors,
    colLabels=column_headers,
    cellLoc="center",
    loc="center",
)

table_props = the_table.properties()
cells = table_props["children"]
for cell in cells:
    cell.set_height(0.14)

for cell in cells:
    cell.set_fontsize(10)

for i, key in enumerate(the_table._cells):
    if key[0] == 0:  # Column labels
        the_table._cells[key].set_fontsize(15)
    if key[1] == 0:  # Row labels
        the_table._cells[key].set_fontsize(15)

plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

plt.savefig("../figures/table.png", dpi=300, bbox_inches="tight")
plt.show()

In [268]:
# import numpy as np
# import pandas as pd
# import plotly.graph_objects as go
# from pathlib import Path
#
# # Ensure output folder exists
# OUT = Path("out")
# OUT.mkdir(exist_ok=True)
#
# # 1) Load & extract primary families
# tox17 = pd.read_csv("../data/processed/toxprot_2017.csv")
# tox25 = pd.read_csv("../data/processed/toxprot_2025.csv")
#
# tox17["Family_2017"] = (
#     tox17["Protein families"].fillna("")
#          .str.split(r"\.", n=1, regex=True).str[0]
# )
# tox25["Family_2025"] = (
#     tox25["Protein families"].fillna("")
#          .str.split(r"[,;]", n=1, regex=True).str[0]
# )
#
# # 2) Merge on Entry (inner join)
# df17 = tox17[["Entry","Family_2017"]]
# df25 = tox25[["Entry","Family_2025"]]
# merged = pd.merge(df17, df25, on="Entry", how="inner")
#
# # 3) Count every transition (self‐flows included)
# flows = (
#     merged
#       .groupby(["Family_2017","Family_2025"])
#       .size()
#       .reset_index(name="value")
# )
#
# # 4) Find global top-10 in each year
# counts_17 = merged["Family_2017"].value_counts()
# counts_25 = merged["Family_2025"].value_counts()
# top10_17  = counts_17.nlargest(10).index.tolist()
# top10_25  = counts_25.nlargest(10).index.tolist()
#
# # 5) Bin into top-10 / Other / nan
# def bin_family(x, top10, tag):
#     if x == "" or pd.isna(x):
#         return f"nan ({tag})"
#     if x in top10:
#         return x
#     return f"Other ({tag})"
#
# flows["old_bin"] = flows["Family_2017"].apply(lambda x: bin_family(x, top10_17, "2017"))
# flows["new_bin"] = flows["Family_2025"].apply(lambda x: bin_family(x, top10_25, "2025"))
#
# binned = (
#     flows
#       .groupby(["old_bin","new_bin"])["value"]
#       .sum()
#       .reset_index()
# )
#
# # 6) Build node list (old on left, new on right)
# old_nodes = top10_17 + ["Other (2017)", "nan (2017)"]
# new_nodes = top10_25 + ["Other (2025)", "nan (2025)"]
# labels   = old_nodes + new_nodes
# idx_map  = {lab: i for i, lab in enumerate(labels)}
#
# binned["source"] = binned["old_bin"].map(idx_map)
# binned["target"] = binned["new_bin"].map(idx_map)
#
# # 7) Compute even y-positions for each node
# n_left  = len(old_nodes)
# n_right = len(new_nodes)
# y_left  = np.linspace(0, 1, n_left)
# y_right = np.linspace(0, 1, n_right)
# node_x  = [0.0]*n_left + [1.0]*n_right
# node_y  = np.concatenate([y_left, y_right]).tolist()
#
# # 8) Create the Sankey with fixed arrangement
# fig = go.Figure(go.Sankey(
#     arrangement="fixed",
#     orientation="h",
#     node=dict(
#         label=labels,
#         x=node_x,
#         y=node_y,
#         pad=10,
#         thickness=30,
#         line=dict(color="black", width=0.5),
#     ),
#     link=dict(
#         source=binned["source"],
#         target=binned["target"],
#         value=binned["value"],
#         color='rgba(0,0,0,0.2)',        # light‐gray fill
#         line=dict(color="black", width=0.5),
#     )
# ))
#
# # 9) Bump fonts for print
# fig.update_layout(
#     title_text="Protein-family flows, 2017 → 2025 (top-10 bins)",
#     font=dict(size=14, family="Arial"),
#     title_font_size=18,
#     width=1200,
#     height=800,
#     margin=dict(l=50, r=50, t=80, b=50),
# )
#
# # Show interactive for quick check
# fig.show()
#
# print(f"Static Sankey exported to: {OUT/'sankey_static.png'}")
